## Capstone Project - The Battle of Neighborhoods
### Introducting new bicycle parking locations

### Content
1 Introduction Section :

    1.1 Discussion and background
    1.2 Problem to be resolved
    1.3 Targeted audience
    
2 Data Section:

    2.1 Required data
    2.2 Data sources
    
3 Methodology:

    3.1 Analysis
    3.2 Clustering
    
4 Results 

5 Disscussion 

6 Conclusion 

### 1 Introduction Section :
[1] Düsseldorf is a city on the River Rhine in western Germany and is the capital city of the state of North Rhine-Westphalia. It is one of the economic centers of the country, and a major city within the Rhine-Ruhr metropolitan area, with a population of almost 600,000.The capital of North Rhine-Westphalia Germany, Dusseldorf is a regional economic powerhouse straddling the banks of the Rhine River. Altstadt is Dusseldorf's lovely old town.
#### 1.1 Discussion and background
[2] Düsseldorf is connected to some national and international cycling paths, including EV15 The Rhine Cycle Route. The city of Düsseldorf is a member of the German North Rhine-Westphalia District, Municipality and City Friends of Pedestrians and Cyclists Working Group, who bestowed upon Düsseldorf the title of "Friend of Cyclists City" in 2007, although the city still has a few gaps in the network of cycle paths in the eyes of many of its citizens.
####  1.2 Problem to be resolved
City have some official bicycle parking spots but most all of them are located on railway station. Can we endorse a location in Duesseldorf to open a new bicycle parking?. New bicycle parking spot will motivate people to go shopping or hangout on bicycles which can reduce the traffic and car parking problem in the city.
#### 1.3 Targeted audience
 The report is aimed primarily at those involved in city planning, sports clubs and students learning data science.

### 2 Data Section:
#### 2.1 Required data
We are going to predict a new bicycle parking location, so a list of existing bicycle location will a perfect point to start. We will need a list of coordinates where currently existing parking are located. For the sake of simplicity we will ignore the number of parking spaces assigned to each bicycle parking place. Also we will only consider bicycle parking which are official assigned from the city. A list of all the zip codes in the city which will be use to gather and cluster the bicycle frendly businesses from each neighbourhood.
The other list we need is the number and type of businesses and avenue located around these existing parking places. Average distance of bicycle parking from avenues can be calculated later. 

#### 2.2 Data sources
A variety of publicly open data about the city of Duesseldorf is available on the city website. We will be using https://opendata.duesseldorf.de/node/40945/download, where a list of all bicycle parking locations with latitudes and longitudes are available as csv file. This data is maintained by the city itself 

In [90]:
import pandas as pd
import numpy as np

In [91]:
#List of parking spaces has been taken from local csv after downloading it from open portal
df = pd.read_csv(r'C:\Users\skhaldy\Downloads\Bike + Ride-Stationen.csv', delimiter=";")
df = df[['LATITUDE','LONGITUDE','Name','PLZ']]
df

,LATITUDE,LONGITUDE,Name,PLZ
0,51.195687,6.768617,Aachener Straße - Aachener Platz,40223
1,51.241592,6.737520,Am Seestern U,40547
2,51.326056,6.775968,An den Kämpen,40489
3,51.329916,6.782346,Angermund S,40489
4,51.328519,6.779024,Auf der Krone,40489
...,...,...,...,...
70,51.209476,6.793592,Volksgarten S,40227
71,51.229640,6.797494,Wehrhahn S,40211
72,51.193260,6.815090,Werstener Dorfstraße,40591
73,51.318502,6.745620,Wittlaer U,40489


from the record we can see there are 75 official bicycle parkings in the city.

We can get a json file about geo data from the following link. This geojson will be use to locate draw the boundries of around every district and give district every district a u
'https://opendata.duesseldorf.de/node/42155/download'

### 3 Methodology
In this venture we will coordinate our endeavors on distinguishing territories of Düsseldorf that have bicycle friendy venues, especially those with low number bicycle parking. We will constrain our investigation to territory around any given parking place. 
In initial step we have gathered the necessary information: area and type of each existing parking inside  from Düsseldorf focus bike+ride places. We have additionally distinguished bicycle friendly businesses (as indicated by Foursquare classification). 
Second step in our investigation will be figuring and investigation of 'bicycle friendly business density' across various territories of Düsseldorf - we will utilize maps to distinguish a couple of promising zones near focus with low number of eateries all in all (and no parking in region) and concentrate on those zones.

#### 3.1 Analysis
In order to get the first impression, we will initialize our analysis by plotting the existing bicycle parking places on a map. For this project we are using folium library to plot the parking coordinates on the map. 

In [94]:
#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

In [95]:
latitude = 51.22172
longitude = 6.77616

df_parking = df
# create map of toronto using latitude and longitude values
map_dus = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(df_parking['LATITUDE'], df_parking['LONGITUDE'], df_parking['Name']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_dus)  
    
map_dus


Next, we are going to start utilizing the Foursquare API to explore the neighborhoods and segment them.
Define Foursquare Credentials and Version

In [96]:
CLIENT_ID = 'TFBCLECRNLWKLIINFW0ICRZ0WJKOCZNRSQFU1XA2OJCI53G2' # your Foursquare ID
CLIENT_SECRET = 'DLMBSFN3L2IOIGIFDCO5MWKU1CZL4N21OWPYGYSLNQYZTOLX' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: TFBCLECRNLWKLIINFW0ICRZ0WJKOCZNRSQFU1XA2OJCI53G2
CLIENT_SECRET:DLMBSFN3L2IOIGIFDCO5MWKU1CZL4N21OWPYGYSLNQYZTOLX


In [97]:
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius

In [98]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):  
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['ParkingName', 
                  'Parking Latitude', 
                  'Parking Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [99]:
dus_parking_venues = getNearbyVenues(names=df_parking['Name'],
                                   latitudes=df_parking['LATITUDE'],
                                   longitudes=df_parking['LONGITUDE']
                                  )

Aachener Straße - Aachener Platz
Am Seestern U
An den Kämpen
Angermund S
Auf der Krone
Barbarossaplatz
Belsenplatz
Benrath S
Benrath, Rathaus
Bilk S
Burgmüllerstraße
Christophstrasse
Derendorf S
Dominikus-Krankenhaus
Eller Mitte
Eller Mitte S
Eller S
Eller Süd S
Flingern S
Flughafen Bf
Flughafen S
Flughafen Terminal S
Flughafen, Halle 7
Flughafenstraße
Flughafenverwaltung
Friedrichstadt S
Froschenteich
Garath S
Gerresheim S
Gerresheim, Krankenhaus
Hamm S
Handweiser U
Heinrich-Heine-Allee U
Hellerhof S
Holthausen
Ickerswarder Straße U
Im Großen Winkel
Jägerstaße U
Jan-Wellem-Platz
Kalkumer Schlossallee
Kettwiger Straße U
Kittelbachstraße
Klemensplatz
Lohausen
Lohweg U
Lörick
Luegplatz U
Maritimplatz
Messe Ost - Stockumer Kirchstraße
Nordstraße
Oberbilk S
Oberrath
Provinzialplatz
Radstation am Hauptbahnhof
Radstation am Hauptbahnhof
Radstation am Hauptbahnhof
Radstation am Rheinufer
Radstation am Rheinufer
Rath S
Rath-Mitte S
Reeser Platz
Reisholz S
Rotthäuser Weg
Staufenplatz
Südfriedho

In [29]:
print(dus_parking_venues.shape)
dus_parking_venues.head()

(1383, 7)


,ParkingName,Parking Latitude,Parking Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Aachener Straße - Aachener Platz,51.195687,6.768617,Unbehaun Eiscafé,51.198613,6.770416,Ice Cream Shop
1,Aachener Straße - Aachener Platz,51.195687,6.768617,Trödelmarkt Aachener Platz,51.197210,6.773744,Flea Market
2,Aachener Straße - Aachener Platz,51.195687,6.768617,Volmerswerther Deich,51.197750,6.763666,Beach
3,Aachener Straße - Aachener Platz,51.195687,6.768617,Hermes,51.194900,6.762892,Greek Restaurant
4,Aachener Straße - Aachener Platz,51.195687,6.768617,Lasertag Evolution Düsseldorf,51.197721,6.772041,Laser Tag


Let's check how many venues were returned for each neighborhood

In [32]:
dus_parking_venues.groupby('ParkingName').count()

,Parking Latitude,Parking Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
ParkingName,,,,,,
Aachener Straße - Aachener Platz,21,21,21,21,21,21
Am Seestern U,17,17,17,17,17,17
An den Kämpen,2,2,2,2,2,2
Angermund S,6,6,6,6,6,6
Auf der Krone,5,5,5,5,5,5
...,...,...,...,...,...,...
Völklinger Straße S,33,33,33,33,33,33
Wehrhahn S,34,34,34,34,34,34
Werstener Dorfstraße,6,6,6,6,6,6


In [42]:
# one hot encoding
dus_onehot = pd.get_dummies(dus_parking_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
dus_onehot['ParkingName'] = dus_parking_venues['ParkingName'] 

# move neighborhood column to the first column
fixed_columns = [dus_onehot.columns[-1]] + list(dus_onehot.columns[:-1])
dus_onehot = dus_onehot[fixed_columns]

dus_onehot.head()

,ParkingName,Airport,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,...,Trattoria/Osteria,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Store,Vietnamese Restaurant,Water Park,Wine Bar,Wine Shop,Yoga Studio,Zoo Exhibit
0,Aachener Straße - Aachener Platz,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Aachener Straße - Aachener Platz,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Aachener Straße - Aachener Platz,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Aachener Straße - Aachener Platz,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Aachener Straße - Aachener Platz,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [46]:
dus_grouped = dus_onehot.groupby('ParkingName').mean().reset_index()
dus_grouped.head()

,ParkingName,Airport,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,...,Trattoria/Osteria,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Store,Vietnamese Restaurant,Water Park,Wine Bar,Wine Shop,Yoga Studio,Zoo Exhibit
0,Aachener Straße - Aachener Platz,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.047619,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Am Seestern U,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,An den Kämpen,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Angermund S,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,Auf der Krone,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0


First, let's write a function to sort the venues in descending order.

In [44]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Now let's create the new dataframe and display the top 10 venues for each neighborhood.

In [83]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['ParkingName']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
parking_venues_sorted = pd.DataFrame(columns=columns)
parking_venues_sorted['ParkingName'] = dus_grouped['ParkingName']

for ind in np.arange(dus_grouped.shape[0]):
    parking_venues_sorted.iloc[ind, 1:] = return_most_common_venues(dus_grouped.iloc[ind, :], num_top_venues)

parking_venues_sorted.head()

,ParkingName,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Aachener Straße - Aachener Platz,Supermarket,Greek Restaurant,Skate Park,Soccer Field,Tram Station,Ice Cream Shop,Park,Financial or Legal Service,Flea Market,Beach
1,Am Seestern U,Hotel,German Restaurant,Italian Restaurant,Gym / Fitness Center,Bus Stop,Chinese Restaurant,Gastropub,Bakery,Ramen Restaurant,Japanese Restaurant
2,An den Kämpen,Business Service,Bus Stop,Zoo Exhibit,Food & Drink Shop,Flower Shop,Flea Market,Financial or Legal Service,Fast Food Restaurant,Farmers Market,Falafel Restaurant
3,Angermund S,Supermarket,Bakery,Pizza Place,Restaurant,Light Rail Station,Zoo Exhibit,Flower Shop,Flea Market,Financial or Legal Service,Fast Food Restaurant
4,Auf der Krone,Light Rail Station,Restaurant,Bed & Breakfast,Bus Stop,Bar,Electronics Store,Food & Drink Shop,Flower Shop,Flea Market,Financial or Legal Service


### 3.2 Clustering
Run k-means to cluster the parkings into 5 clusters.

In [67]:
# import k-means from clustering stage
from sklearn.cluster import KMeans

In [82]:
# set number of clusters
kclusters = 5

dus_grouped_clustering = dus_grouped.drop('ParkingName', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(dus_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([1, 1, 2, 3, 0, 1, 1, 1, 1, 1])

Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood

In [84]:
# add clustering labels
parking_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

In [79]:
dus_merged = df

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
dus_merged = dus_merged.join(parking_venues_sorted.set_index('ParkingName'), on='Name')

dus_merged.head() # check the last columns!

,LATITUDE,LONGITUDE,Name,PLZ,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,51.195687,6.768617,Aachener Straße - Aachener Platz,40223,1,Supermarket,Greek Restaurant,Skate Park,Soccer Field,Tram Station,Ice Cream Shop,Park,Financial or Legal Service,Flea Market,Beach
1,51.241592,6.737520,Am Seestern U,40547,1,Hotel,German Restaurant,Italian Restaurant,Gym / Fitness Center,Bus Stop,Chinese Restaurant,Gastropub,Bakery,Ramen Restaurant,Japanese Restaurant
2,51.326056,6.775968,An den Kämpen,40489,1,Business Service,Bus Stop,Zoo Exhibit,Food & Drink Shop,Flower Shop,Flea Market,Financial or Legal Service,Fast Food Restaurant,Farmers Market,Falafel Restaurant
3,51.329916,6.782346,Angermund S,40489,2,Supermarket,Bakery,Pizza Place,Restaurant,Light Rail Station,Zoo Exhibit,Flower Shop,Flea Market,Financial or Legal Service,Fast Food Restaurant
4,51.328519,6.779024,Auf der Krone,40489,1,Light Rail Station,Restaurant,Bed & Breakfast,Bus Stop,Bar,Electronics Store,Food & Drink Shop,Flower Shop,Flea Market,Financial or Legal Service


In [85]:
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

In [86]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(dus_merged['LATITUDE'], dus_merged['LONGITUDE'], dus_merged['Name'], dus_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [100]:
dus_merged.loc[dus_merged['Cluster Labels'] == 0, dus_merged.columns[[2] + list(range(5, dus_merged.shape[1]))]]

,Name,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
19,Flughafen Bf,Platform,Bus Stop,Rental Car Location,Diner,Zoo Exhibit,Flower Shop,Flea Market,Financial or Legal Service,Fast Food Restaurant,Farmers Market
20,Flughafen S,Platform,Bus Stop,Rental Car Location,Diner,Zoo Exhibit,Flower Shop,Flea Market,Financial or Legal Service,Fast Food Restaurant,Farmers Market


In [102]:
dus_merged.loc[dus_merged['Cluster Labels'] == 1, dus_merged.columns[[2] + list(range(5, dus_merged.shape[1]))]]

,Name,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Aachener Straße - Aachener Platz,Supermarket,Greek Restaurant,Skate Park,Soccer Field,Tram Station,Ice Cream Shop,Park,Financial or Legal Service,Flea Market,Beach
1,Am Seestern U,Hotel,German Restaurant,Italian Restaurant,Gym / Fitness Center,Bus Stop,Chinese Restaurant,Gastropub,Bakery,Ramen Restaurant,Japanese Restaurant
2,An den Kämpen,Business Service,Bus Stop,Zoo Exhibit,Food & Drink Shop,Flower Shop,Flea Market,Financial or Legal Service,Fast Food Restaurant,Farmers Market,Falafel Restaurant
4,Auf der Krone,Light Rail Station,Restaurant,Bed & Breakfast,Bus Stop,Bar,Electronics Store,Food & Drink Shop,Flower Shop,Flea Market,Financial or Legal Service
5,Barbarossaplatz,Italian Restaurant,Bar,Café,Supermarket,Trattoria/Osteria,French Restaurant,Pub,Plaza,Bakery,Music Venue
6,Belsenplatz,Bar,Café,Plaza,Gastropub,Italian Restaurant,Bakery,Greek Restaurant,Brewery,Burger Joint,French Restaurant
7,Benrath S,Café,Museum,Frozen Yogurt Shop,Palace,Bank,Drugstore,Greek Restaurant,Restaurant,Supermarket,Hockey Rink
8,"Benrath, Rathaus",Ice Cream Shop,Japanese Restaurant,Palace,Bus Line,Mediterranean Restaurant,Tram Station,Bakery,Hotel,Fast Food Restaurant,Duty-free Shop
9,Bilk S,Pub,Café,Middle Eastern Restaurant,Bar,Taverna,BBQ Joint,Fast Food Restaurant,Mexican Restaurant,Smoke Shop,Bistro
10,Burgmüllerstraße,Middle Eastern Restaurant,Tram Station,Ice Cream Shop,Drugstore,Fast Food Restaurant,Sports Club,Greek Restaurant,Bakery,Athletics & Sports,Plaza


In [103]:
dus_merged.loc[dus_merged['Cluster Labels'] == 2, dus_merged.columns[[2] + list(range(5, dus_merged.shape[1]))]]

,Name,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
3,Angermund S,Supermarket,Bakery,Pizza Place,Restaurant,Light Rail Station,Zoo Exhibit,Flower Shop,Flea Market,Financial or Legal Service,Fast Food Restaurant
14,Eller Mitte,Supermarket,Drugstore,Light Rail Station,Bakery,Pizza Place,Brewery,Shoe Store,Metro Station,Zoo Exhibit,Flea Market
15,Eller Mitte S,Supermarket,Drugstore,Light Rail Station,Bakery,Pizza Place,Brewery,Shoe Store,Metro Station,Zoo Exhibit,Flea Market
16,Eller S,Light Rail Station,Mini Golf,Garden,Supermarket,Zoo Exhibit,Flower Shop,Flea Market,Financial or Legal Service,Fast Food Restaurant,Farmers Market
17,Eller Süd S,Light Rail Station,Bakery,Shoe Store,Metro Station,Supermarket,Drugstore,Zoo Exhibit,Flower Shop,Flea Market,Financial or Legal Service
27,Garath S,Supermarket,Light Rail Station,Pizza Place,Pharmacy,Outlet Mall,Zoo Exhibit,Flea Market,Financial or Legal Service,Fast Food Restaurant,Farmers Market
33,Hellerhof S,Supermarket,Light Rail Station,Bakery,Zoo Exhibit,Food & Drink Shop,Flower Shop,Flea Market,Financial or Legal Service,Fast Food Restaurant,Farmers Market
34,Holthausen,Supermarket,Drugstore,Hotel,Middle Eastern Restaurant,Duty-free Shop,Food & Drink Shop,Flower Shop,Flea Market,Financial or Legal Service,Fast Food Restaurant
36,Im Großen Winkel,Supermarket,Bakery,Pizza Place,Restaurant,Light Rail Station,Zoo Exhibit,Flower Shop,Flea Market,Financial or Legal Service,Fast Food Restaurant
37,Jägerstaße U,Supermarket,Metro Station,Drugstore,Light Rail Station,Food & Drink Shop,Zoo Exhibit,Flower Shop,Flea Market,Financial or Legal Service,Fast Food Restaurant


In [104]:
dus_merged.loc[dus_merged['Cluster Labels'] == 3, dus_merged.columns[[2] + list(range(5, dus_merged.shape[1]))]]

,Name,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
41,Kittelbachstraße,Restaurant,Light Rail Station,Hotel,Zoo Exhibit,Flower Shop,Flea Market,Financial or Legal Service,Fast Food Restaurant,Farmers Market,Falafel Restaurant
62,Rotthäuser Weg,Restaurant,Cosmetics Shop,Hotel,Jewelry Store,Stables,Zoo Exhibit,Drugstore,Flea Market,Financial or Legal Service,Fast Food Restaurant


#### 4. Discussion

Our investigation shows that in spite of the fact that there is an incredible number of bicycle parking in Düsseldorf, there are pockets of low parking thickness genuinely near downtown area (Werstener Dorfstrasse). Most noteworthy centralization of parkings was distinguished north and west from the city

#### 5. Results

As shown in the figure above a new bike parking station between ‘Werstener Dorfstrasse’ and ‘Eller Süd’ may provide an efficient bicycle parking place for the for the riders. This new parking place will tightly couple the bicycle friendly parking cluster (cluster 2). 

##### 6 Conclusion

In this report for the sake of simplicity and learning we have taken a very simple approach to find a new location for the parking. For further optimization we can consider the distance between city center and the parking spaces and add it to our dataset to have a even better clustering. 
Further more boundaries of the city districts can be user in relation to bicycle friendly areas to create a heat map of the city. Such a heat map can provide a better understanding of the unfilled gaps of parking between different districts

#### references
1.	https://en.wikipedia.org/wiki/D%C3%BCsseldorf
2.	https://en.wikivoyage.org/wiki/D%C3%BCsseldorf
